In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 53 kB/s 
     |████████████████████████████████| 199 kB 66.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b2fb199f7697d2bd1048e44c3119dd96c482a6e5a3832f044d65885502f36869
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
import pyspark as spark
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import glob
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
df = pd.read_csv('./drive/MyDrive/df_modeling.csv')
df_test = pd.read_csv('./drive/MyDrive/final_test_data.csv',encoding='utf-8')
df.head()

spark_df = sqlContext.createDataFrame(df)
spark_df_test = sqlContext.createDataFrame(df_test)

In [ ]:
from pyspark.ml.feature import VectorAssembler

# 1. Train/Test 데이터로 분할
#train_df, test_df = spark_df.randomSplit(weights=[0.8, 0.2], seed=42)
train_df = spark_df
test_df = spark_df_test


# 2. Feature Vector로 만들기
ftr_columns = ['max_rpm','s20', 's30', 's40', 's50', 's60', 's70', 's80','s90','s100','s110', 's120','s130','s140','s150']
vec_assembler = VectorAssembler(inputCols=ftr_columns, outputCol='features') #features에는 하나의 칼럼.

train_ftr_vec = vec_assembler.transform(train_df)
test_ftr_vec = vec_assembler.transform(test_df)  # 테스트 데이터에 대해도 동일한 객체 사용
sparktrain_ftr_vec = vec_assembler.transform(test_df)

print('# 학습 원본 데이터:')
train_df.limit(5).show()

print('# Feature Vectorization 후 학습 데이터:')
train_ftr_vec.limit(5).show()

# 학습 원본 데이터:
+-----+----------+-----------+----------+-----------+-------+---+---+---+---+---+---+---+---+----+----+----+----+----+----+-----------------------------+----+-------------+------------------+------------------+------------------+-------------------+-------------------+--------+------------------+-----------------+------+--------------------+--------------------+
|index|     s_lat|      s_lng|     e_lat|      e_lng|max_rpm|s20|s30|s40|s50|s60|s70|s80|s90|s100|s110|s120|s130|s140|s150|                         path| fee|duration(min)|     root_dist(km)|      med_est_dist|              diff|                pc1|                pc2|clusters|          time_sum|        avg_speed|target|                   s|                   e|
+-----+----------+-----------+----------+-----------+-------+---+---+---+---+---+---+---+---+----+----+----+----+----+----+-----------------------------+----+-------------+------------------+------------------+------------------+-------------------+--------

#랜덤포레스트

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

# 모델 정의
dt_clf = RandomForestClassifier(featuresCol='features', labelCol='target', numTrees=10)

# 학습
dt_model = dt_clf.fit(train_ftr_vec)

# 학습, 테스트 데이터에 대해 예측
train_pred = dt_model.transform(train_ftr_vec)
test_pred = dt_model.transform(test_ftr_vec)


In [ ]:
train_pred.show()

+-----+----------+-----------+----------+-----------+-----------+---+---+---+---+---+---+---+---+----+----+----+----+----+----+---------------------------------+----+-------------+------------------+------------------+------------------+-------------------+-------------------+--------+------------------+------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|index|     s_lat|      s_lng|     e_lat|      e_lng|    max_rpm|s20|s30|s40|s50|s60|s70|s80|s90|s100|s110|s120|s130|s140|s150|                             path| fee|duration(min)|     root_dist(km)|      med_est_dist|              diff|                pc1|                pc2|clusters|          time_sum|         avg_speed|target|                   s|                   e|            features|       rawPrediction|         probability|prediction|
+-----+----------+-----------+----------+-----------+-----------+---+---+---+---+---+---+---+---+----+--

In [ ]:
test_pred.show()

+----------+-------+---+---+---+---+---+---+---+---+----+----+----+----+----+----+-------------+----------+---------------+-----+-------------+------------------+--------------------+--------------------+--------------------+----------+
|Unnamed: 0|max_rpm|s20|s30|s40|s50|s60|s70|s80|s90|s100|s110|s120|s130|s140|s150|        path1|     path2|          path3|  fee|duration(min)|     root_dist(km)|            features|       rawPrediction|         probability|prediction|
+----------+-------+---+---+---+---+---+---+---+---+----+----+----+----+----+----+-------------+----------+---------------+-----+-------------+------------------+--------------------+--------------------+--------------------+----------+
|         0| 4041.4|118|134| 87|119|  0|  8|  0|  2|   8|   8|   0|   0|   0|   0| "마곡중앙8로|  공항대로|        하늘길"| 6000|           13|               4.4|[4041.4,118.0,134...|[0.59858974358974...|[0.05985897435897...|       1.0|
|         1|3778.68|289|305|138|170| 95|103|  0|  2|   8|   8|  

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='target', metricName='accuracy')

train_acc = evaluator.evaluate(train_pred)
#test_acc = evaluator.evaluate(test_pred)

print('Train Accuracy:', train_acc)
#print('Test Accuracy:', test_acc)

Train Accuracy: 0.86391711496314


#결정트리

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

# 모델 정의
dt_clf = DecisionTreeClassifier(featuresCol='features', labelCol='target', maxDepth=10)

# 학습
dt_model = dt_clf.fit(train_ftr_vec)

# 학습, 테스트 데이터에 대해 예측
train_pred = dt_model.transform(train_ftr_vec)
test_pred = dt_model.transform(test_ftr_vec)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='target', metricName='accuracy')

train_acc = evaluator.evaluate(train_pred)
#test_acc = evaluator.evaluate(test_pred)

print('Train Accuracy:', train_acc)
#print('Test Accuracy:', test_acc)

Train Accuracy: 0.9735006973500697


In [ ]:
test_pred.show()

+----------+-------+---+---+---+---+---+---+---+---+----+----+----+----+----+----+-------------+----------+---------------+-----+-------------+------------------+--------------------+------------------+--------------------+----------+
|Unnamed: 0|max_rpm|s20|s30|s40|s50|s60|s70|s80|s90|s100|s110|s120|s130|s140|s150|        path1|     path2|          path3|  fee|duration(min)|     root_dist(km)|            features|     rawPrediction|         probability|prediction|
+----------+-------+---+---+---+---+---+---+---+---+----+----+----+----+----+----+-------------+----------+---------------+-----+-------------+------------------+--------------------+------------------+--------------------+----------+
|         0| 4041.4|118|134| 87|119|  0|  8|  0|  2|   8|   8|   0|   0|   0|   0| "마곡중앙8로|  공항대로|        하늘길"| 6000|           13|               4.4|[4041.4,118.0,134...| [0.0,0.0,2.0,0.0]|   [0.0,0.0,1.0,0.0]|       2.0|
|         1|3778.68|289|305|138|170| 95|103|  0|  2|   8|   8|   0|   0|